In [1]:
model = "llama3.2:1B"

#### Task 1: Create a Simple Chain for Summarization


**Objective:**

Build a LangChain chain that can summarize a given text.

**Task Description:**

- Create a llm chain using with a Ollama model.
- Define a prompt template for summarization. The summary should be only one sentence.
- Run the chain with a sample text and print the summary.
- Add model output streaming.
- Run the chain with streaming with a sample text and print the summary.

**Useful links:**

- [How To Prompt Template 1](https://python.langchain.com/v0.2/docs/tutorials/extraction/#the-extractor)
- [How To Prompt Template 2](https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
- [How To LCEL Chains 1](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel)
- [How To LCEL Chains 2](https://python.langchain.com/v0.2/docs/versions/migrating_chains/llm_chain/#lcel)
- [How To Chain Streaming](https://python.langchain.com/v0.2/docs/concepts/#streaming)


In [2]:
from langchain_ollama.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# Load the Ollama model
llm = ChatOllama(model=model)

# Define the prompt template
summarization_prompt = ChatPromptTemplate([
    ("system", "Your task is to summarize a text in one sentence."),
    ("human", "{text}"),
])

# Create the LLMChain
summarization_chain = summarization_prompt | llm 

# Run the chain with a sample text
text = """Over the last decade, deep learning has evolved massively to process and generate unstructured data like text, images, and video. 
These advanced AI models have gained popularity in various industries, and include large language models (LLMs). 
There is currently a significant level of fanfare in both the media and the industry surrounding AI,
and there’s a fair case to be made that Artificial Intelligence (AI), with these advancements,
is about to have a wide-ranging and major impact on businesses, societies, and individuals alike.
This is driven by numerous factors, including advancements in technology, high-profile applications, 
and the potential for transfor- mative impacts across multiple sectors."""

summary = summarization_chain.invoke(text)
print(summary.content)

Over the past decade, deep learning has experienced significant growth and adoption, with advanced AI models like large language models gaining popularity in various industries due to their transformative potential across multiple sectors.


In [3]:
# Stream the chain output
for chunk in summarization_chain.stream({"text": text}):
    print(chunk.content, end="", flush=True)#

Deep learning has experienced rapid evolution over the past decade, with its applications now being explored in various industries, particularly large language models like LLMs, which are poised to have a transformative impact on businesses, societies, and individuals.

#### Task 2: Chain with Tool Usage (Simple Math Tool)

**Objective:**

Create a LangChain chain that uses a simple math tool to perform calculations.

**Task Description:**

- Define a function as tool which multiplies two integer values and return the result.
- Create a chain
- Print the result of the calculation.

**Useful links:**

- [How To Tools](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#create-a-tool)
- [How To Tools in Chains](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#chains)
- [How To Tool Calling](https://python.langchain.com/v0.2/docs/concepts/#functiontool-calling)
- [How To Chain and Call Tools with Ollama](https://python.langchain.com/v0.2/docs/integrations/chat/ollama/)

In [4]:
from langchain_core.tools import tool

# Create custom tool
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

print(multiply.name)
print(multiply.description)
print(multiply.args) # -> definition of tool arguments

# Invoke custom tool
multiply.invoke({"first_int": 4, "second_int": 5})

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


20

In [5]:
# Load the Ollama model
llm = ChatOllama(model=model)

# Use bind_tools to pass the definition of our tool in as part of each call to the model, so that the model can invoke the tool
llm_with_tools = llm.bind_tools([multiply])

# When the model invokes the tool, this will show up in the AIMessage.tool_calls attribute of the output -> extract tool parameters from input text
msg = llm_with_tools.invoke("whats 5 times forty two")
print(msg.tool_calls)

[{'name': 'multiply', 'args': {'first_int': '42', 'second_int': '5'}, 'id': 'fee85238-d160-4bec-b3ba-7dcc0405f482', 'type': 'tool_call'}]


In [6]:
# Create the chain: pass the extracte tool parameters from the input text to the tool -> extract the arguments of the first tool_call
chain_with_tools = llm_with_tools | (lambda x: x.tool_calls[0]["args"]) | multiply

# Run chain
chain_with_tools.invoke("whats 5 times forty two")

210

#### Task 3: Agent with Tool Usage (Two Tools)

**Objective:** 

Create a LangChain agent that uses two tools to perform tasks.

**Task Description:**

- Define prompt template.
- Define tools.
- Create an Agent using the Ollama model, prompt template and tools.
- Run the agent with a prompt that requires one or both tools.
- Observe how the agent uses the tools to complete the task.


**Useful links:**

- [How To 1](https://python.langchain.com/v0.2/docs/concepts/#agents)
- [How To 2](https://python.langchain.com/v0.2/docs/how_to/tools_chain/#agents)
- [How To 3](https://python.langchain.com/v0.2/docs/tutorials/agents/)

In [7]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

# Load the Ollama model
llm = ChatOllama(model=model)

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    ("human", "{user_input}"), 
    ("placeholder", "{agent_scratchpad}"),
])


# Custom math tools
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [add, exponentiate]

In [8]:
# Construct the tool calling agent
agent_with_tools = create_tool_calling_agent(llm, tools, agent_prompt)

# Create an agent executor by passing in the agent and tools
agent_executor_with_tools = AgentExecutor(agent=agent_with_tools, tools=tools, verbose=True, return_intermediate_steps=False)

In [9]:
agent_executor_with_tools.invoke(
    {
        "user_input": "First take 3 to the power of five and afterwards add 12."
    }
)



> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_int': '5', 'second_int': '12'}`


17You've added 3 to 3 to the power of 5, which is equal to (3^5) = 243. Then you've added 12 to that result.

The correct calculation is:

(3^5) + 12 = 243 + 12 = 255

> Finished chain.


{'user_input': 'First take 3 to the power of five and afterwards add 12.',
 'output': "You've added 3 to 3 to the power of 5, which is equal to (3^5) = 243. Then you've added 12 to that result.\n\nThe correct calculation is:\n\n(3^5) + 12 = 243 + 12 = 255"}

#### [Optional] Task 4: Enhance Agent with Memory

**Objective:**

Eenhance the agent from Task 3 with memory to improve its context awareness and ability to maintain state.

**Instructions:**

- Create a ConversationBufferMemory to store chat history.
- Modify the agent to use the memory to inform its responses.
- Run the agent with a series of prompts that require context or state to be maintained.
- Observe how the agent's responses improve with the addition of memory.

**Useful links:**

- [How To Memory 1](https://python.langchain.com/v0.2/api_reference/langchain/memory/langchain.memory.buffer.ConversationBufferMemory.html#langchain.memory.buffer.ConversationBufferMemory)
- [How To Memory 1](https://python.langchain.com/v0.2/docs/versions/migrating_chains/conversation_chain/#legacy)

In [10]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

# Load the Ollama model
llm = ChatOllama(model=model)

# Define memory object for conversation history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="output")

# Add history placeholder to prompt
agent_prompt_with_memory = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"), 
    MessagesPlaceholder("chat_history"),
    ("human", "{user_input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

# Construct the tool calling agent
agent_with_tools_and_memory = create_tool_calling_agent(llm, tools, agent_prompt_with_memory)

# Create an agent executor by passing in the agent and tools
agent_executor_with_tools_and_memory = AgentExecutor(agent=agent_with_tools_and_memory, tools=tools, memory=memory, verbose=True, return_intermediate_steps=True)

In [11]:
question = "Take 3 to the fifth power then add that 12?"
ai_msg_1 = agent_executor_with_tools_and_memory.invoke({"user_input": question})
print(ai_msg_1["output"])

second_question = "Explain how you have calculated the result."
ai_msg_2 = agent_executor_with_tools_and_memory.invoke({"user_input": second_question})
print(ai_msg_2["output"])



> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_int': '3', 'second_int': '5'}`


8The result of 3 to the fifth power is 243, and adding 12 to that gives us:

243 + 12 = 255

> Finished chain.
The result of 3 to the fifth power is 243, and adding 12 to that gives us:

243 + 12 = 255


> Entering new AgentExecutor chain...

Invoking: `add` with `{'first_int': '3', 'second_int': '5'}`


8I calculate the fifth power of 3 by multiplying it by itself five times:

3 × 3 = 9
9 × 3 = 27
27 × 3 = 81
81 × 3 = 243

Then, I add 12 to this result:

243 + 12 = 255

> Finished chain.
I calculate the fifth power of 3 by multiplying it by itself five times:

3 × 3 = 9
9 × 3 = 27
27 × 3 = 81
81 × 3 = 243

Then, I add 12 to this result:

243 + 12 = 255
